# Supervised data compression via linear discriminant analysis

Linear Discriminant Analysis (LDA) can be used as a technique for feature extraction to increase the computational efficiency and reduce the degree of overfitting due to the curse of dimensionality in non-regularized models.

Whereas PCA attempts to find the orthogonal component axes of maximum variance in a dataset, the goal in LDA is to find the feature subspace that optimizes class separability.

## Principal component analysis versus linear discriminant analysis

The aim of both is to reduce the number of dimensions in a dataset, PCA is an unsupervised algorithm whereas LDA is supervised. We might think that LDA is superior, however PCA tends to result in better classification results in an image recognition task in certain cases.

The following figure summarizes the concept of LDA for a two-class problem.

<img src="images/lda_concept.jpeg" alt="PCA orthogonal axes" title="PCA orthogonal axes" height="300" width="450">

A linear discriminant, as shown on the x-axis (LD 1), would separate the two normal distributed classes well. Although the exemplary linear discriminant shown on the y-axis (LD 2) captures a lot of the variance in the dataset, it would fail as a good linear discriminant since it does not capture any of the class-discriminatory information.

One assumption in LDA is that the data is normally distributed. Also, we assume that the classes have identical covariance matrices and that the samples are statistically independent of each other. However, even if one or more of those assumptions are (slightly) violated, LDA for dimensionality reduction can still work reasonably well 

### The inner working

Before we dive into the code implementation, let's briefly summarize the main steps that are required to perform LDA:

1. Standardize the $d$-dimensional dataset ($d$ is the number of features).
2. For each class, compute the $d$-dimensional mean vector.
3. Construct the between-class scatter matrix $S_B$ and the within-class scatter matrix $S_w$. 
4. Compute the eigenvectors and corresponding eigenvalues of the matrix $S_w^{-1} S_B$.
5. Sort the eigenvalues by decreasing order to rank the corresponding eigenvectors.
6. Choose the $k$ eigenvectors that correspond to the $k$ largest eigenvalues to construct a $d \times k$-dimensional transformation matrix $W$; the eigenvectors are the columns of this matrix.
7. Project the samples onto the new feature subspace using the transformation matrix $W$.

#### Computing the scatter matrices

After load and standardized the features, proceed with the calculation of the mean vectors, which we will use to construct the within-class scatter matrix and between-class scatter matrix, respectively. Each mean vector $m_i$ stores the mean feature value $\mu_m$ with respect to the samples of class **i**:

\begin{equation*}
m_i = \frac{1}{n_i} \sum_{x \in D_i}^c x_m
\end{equation*}

This results in three mean vectors:

\begin{equation*}
\begin{matrix}
m_i = \begin{bmatrix}
\mu_i, alcohol \\
\mu_i, \text{malic acid} \\
\vdots \\
\mu_i, proline
\end{bmatrix} & i \in \left\{ 1, 2, 3 \right\} \\
\end{matrix}
\end{equation*}

In [ ]:
# load wine data

In [ ]:
# separate training and test data (70, 30)

# standardize the features

In [ ]:
#  calculates and displays the mean vector for each of the three classes in the standardized training datase

Using the mean vectors, we can now compute the within-class scatter matrix $S_w$:

\begin{equation*}
S_w = \sum_{i=1}^c S_i
\end{equation*}

This is calculated by summing up the individual scatter matrices $S_i$ of each individual class $i$:

\begin{equation*}
S_i = \sum_{x \in D_i}^c \left( x - m_i \right) \left( x - m_i \right)^T
\end{equation*}

In [ ]:
# iterates through classes, calculates the scatter matrix for each, and accumulates them into the within-class scatter matrix (S_W)

The assumption that we are making when we are computing the scatter matrices is that the class labels in the training set are uniformly distributed. However, if we print the number of class labels, we see that this assumption is violated:

In [ ]:
# show class distribution

Thus, we want to scale the individual scatter matrices $S_i$ before we sum them up as scatter matrix $S_w$. When we divide the scatter matrices by the number of class-samples $n_i$, we can see that computing the scatter matrix is in fact the same as computing the covariance matrix $\Sigma_i$ —the covariance matrix is a normalized version of the scatter matrix:

\begin{equation*}
\Sigma_i = \frac{1}{n_i}S_w = \frac{1}{n_i} \sum_{x \in D_i}^c \left( x - m_i \right) \left( x - m_i \right)^T
\end{equation*}

In [ ]:
# calculates the within-class scatter matrix (S_W) by accumulating the covariance matrices for each class in the training data

After we computed the scaled within-class scatter matrix (or covariance matrix), we can move on to the next step and compute the between-class scatter matrix $S_B$:

\begin{equation*}
S_B = \sum_{i=1}^c n_i \left( m_i - m \right) \left( m_i - m \right)^T
\end{equation*}

Here, $m$ is the overall mean that is computed, including samples from all classes:

In [ ]:
# calculates the between-class scatter matrix (S_B) by iterating through class means, considering class sizes, and accumulating the weighted scatter matrices

## Selecting linear discriminants for the new feature subspace

The remaining steps of the LDA are similar to the steps of the PCA. However, instead of performing the eigendecomposition on the covariance matrix, we solve the generalized eigenvalue problem of the matrix $S_w^{-1} S_B$:

In [ ]:
# calculate eigenvalues and eigenvecs

In [ ]:
# sort the eigenvalues in descending order

In LDA, the number of linear discriminants is at most $c−1$, where c is the number of class labels, since the in-between scatter matrix $S_B$ is the sum of c matrices with rank 1 or less. We can indeed see that we only have two nonzero eigenvalues (the eigenvalues 3-14 are not exactly zero, but this is due to the floating point arithmetic in NumPy).

To measure how much of the class-discriminatory information is captured by the linear discriminants (eigenvectors), let's plot the linear discriminants by decreasing eigenvalues similar to the explained variance plot that we created in the PCA section.

In [ ]:
# show individual and cumulative discrimilability

Let's now stack the two most discriminative eigenvector columns to create the transformation matrix $W$:

## Projecting samples onto the new feature space

Using the transformation matrix $W$ that we created in the previous subsection, we can now transform the training dataset by multiplying the matrices:

\begin{equation*}
X^{\prime} = XW
\end{equation*}

In [ ]:
# Projecting samples onto the new feature space

## LDA via scikit-learn

Now, let's look at the LDA class implemented in scikit-learn:

In [ ]:
# LDA via scikit-learn

In [ ]:
# ler's see how it works with logistic regression classifier

By lowering the regularization strength, we could probably shift the decision boundaries so that the logistic regression model classifies all samples in the training dataset correctly. However, and more importantly, let us take a look at the results on the test set: